# Atividade de Laboratório 1
### Integrantes
- Leonardo Mallmann
- Caetano Muller
- Euzébio Hensel

In [41]:
import urllib.request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError, ContentTooShortError
import re
import csv

In [42]:
def download(url, user_agent='wswp', num_retries=2, charset='utf-8'):
  print('Downloading:', url)
  request = urllib.request.Request(url)
  request.add_header('User-agent', user_agent)
  try:
    resp = urllib.request.urlopen(request)
    cs = resp.headers.get_content_charset()
    if not cs:
        cs = charset
    html = resp.read().decode(cs)
  except (URLError, HTTPError, ContentTooShortError) as e:
    print('Download error:', e.reason)
  return html

In [65]:
def scrap(html_content):
  soup = BeautifulSoup(html_content, 'html5lib')
  rows = soup.select('table tbody tr')
  field_names = []
  field_values = []
  for row in rows:
    c = row.select('.w2p_fl label');
    field_names.append(c[0].string.strip().replace(':', ''));
    v = row.select('.w2p_fw');
    if not v[0].string:
      if len(re.findall('<img', str(v[0]))) != 0:
        img_src = v[0].select('img')[0];
        field_values.append(img_src['src'])
        print(img_src['src'])
        continue
      if len(re.findall('<div', str(v[0]))) != 0:
        a_links = v[0].select('div a');
        neighbours = [a_tag.string.strip() for a_tag in a_links]
        field_values.append(neighbours);
        print(neighbours)
        continue
    field_values.append(v[0].string)
    print(v[0].string)
    

### Exemplo de inserção no CSV

In [ ]:
with open('names.csv', 'w', newline='') as csvfile:
  fieldnames = ['first_name', 'last_name']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

  ingre = ['Baked', 'Beans']

  writer.writeheader()
  writer.writerow({name: ing for name, ing in zip(fieldnames,ingre)}) # or dict(zip(keys, values))
  writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
  writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})

with open('names.csv', 'r', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    print(row['first_name'])

In [44]:
def crawl_sitemap(url):
  sitemap = download(url)
  links = re.findall('\d(\/.*)<\/loc>', sitemap)
  for index, link in enumerate(links):
    html = download(f'http://localhost:8000{link}')
    scrap(html)
    if index==0:
      break;

In [67]:
crawl_sitemap('http://localhost:8000/places/static/sitemap.xml');

Downloading: http://localhost:8000/places/static/sitemap.xml
Downloading: http://localhost:8000/places/default/view/Afghanistan-1
647500 square kilometres
29121286
AF
Afghanistan
Kabul
AS
.af
AFN
Afghani
93
None
None
fa-AF,ps,uz-AF,tk
['TM', 'CN', 'IR', 'TJ', 'PK', 'UZ']
